In [1]:
import numpy as np

def listtoarray2(data): # list의 각 element의 행이 1일 때 사용. Vertical 방향으로 stack된 형태의 array 출력됨.
    data_out = []
    if len(data) != 1:
        for count in range(len(data)):
            if count == 0:
                data_out = data[count]
            else:
                data_out = np.vstack((data_out, data[count]))
    else:
        data_out = data[0]
    return(data_out)

def parsingdata(data, check_num):
    temp_raw = np.array(data)
    data_out = []
    temp = np.zeros((1,temp_raw[0,0,:].shape[0]))
    for i in range(temp_raw.shape[1]):
        if np.float64(temp_raw[0,i,0]).item() == check_num:
            temp_comp = np.delete(temp, 0, 0)
            data_out.append(temp_comp)
            temp = np.zeros((1,temp_raw[0,0,:].shape[0]))
            continue
        else:
            temp2 = np.reshape(temp_raw[0,i,:],(1,-1))
            temp = np.append(temp,temp2,axis=0)
            
    return(data_out)

In [2]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import load_model
import pandas as pd

input_train = [np.array(pd.read_csv('Access_Gesturetraining_sensor.csv', header=None))]
output_train = [np.array(pd.read_csv('Access_Gesturetraining_class.csv', header=None))]

# parsing with certain number
check_num = 10000.0

# for training data
input_trainset = parsingdata(input_train, check_num)
output_trainset = parsingdata(output_train, check_num)
output_trainset = listtoarray2(output_trainset) # model.fit의 입력으로 list 타입이 아닌 np.array 타입으로 넣어야 함.

# Zero-padding
input_trainset = sequence.pad_sequences(input_trainset, dtype='float32', padding='pre')

In [3]:
from keras.initializers import glorot_normal
from keras.models import Model, load_model
from keras.layers import Input, Dense, Flatten, Reshape, concatenate, Multiply, Add, LSTM, Lambda
from keras.optimizers import Nadam, Adamax, Adam

import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import time
import math
import os
import csv

def Gesture_classification(columns_size):
    with tf.device('/gpu:0'):
        sensor_input = Input(shape=(None,columns_size))
        hidden1 = LSTM(64, return_sequences=True)(sensor_input)
        hidden2 = LSTM(64)(hidden1)
        
        y1 = Dense(64, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(hidden2)
        y2 = Dense(64, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(y1)
        y3 = Dense(64, activation = 'relu', kernel_initializer=glorot_normal(seed=1))(y2)

        Gesture_output = Dense(11, activation = 'softmax', kernel_initializer=glorot_normal(seed=1))(y3)

        Gesture_model = Model(inputs = sensor_input, outputs = Gesture_output)
        
    return(Gesture_model)

def weightexport_LSTM(model, layer_ind):
    for i in range(len(layer_ind)):
        LSTM_weight_x = model.layers[layer_ind[i]].get_weights()[0]
        LSTM_weight_h = model.layers[layer_ind[i]].get_weights()[1]
        LSTM_bias = model.layers[layer_ind[i]].get_weights()[2]
        pd.DataFrame(LSTM_weight_x).to_csv('LSTM_weight_x'+str(layer_ind[i])+'.csv', header = False, index = False)
        pd.DataFrame(LSTM_weight_h).to_csv('LSTM_weight_h'+str(layer_ind[i])+'.csv', header = False, index = False)
        pd.DataFrame(LSTM_bias).to_csv('LSTM_bias'+str(layer_ind[i])+'.csv', header = False, index = False)
        
def weightexport_Dense(model, layer_ind):
    for i in range(len(layer_ind)):
        Dense_weight = model.layers[layer_ind[i]].get_weights()[0]
        Dense_bias = model.layers[layer_ind[i]].get_weights()[1]
        pd.DataFrame(Dense_weight).to_csv('Dense_weight'+str(layer_ind[i])+'.csv', header = False, index = False)
        pd.DataFrame(Dense_bias).to_csv('Dense_bias'+str(layer_ind[i])+'.csv', header = False, index = False)

In [4]:
Gesture_model = Gesture_classification(10) # model의 입력은 사용된 센서 개수를 의미함.
Gesture_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
Gesture_model.summary()

# model compile 후 실제로 학습되는 과정: model.fit
Gesture_model.fit(input_trainset, output_trainset, epochs=10)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 10)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 64)          19200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 11)                715   

In [15]:
print(input_trainset)

[[[ 0.       0.       0.      ...  0.       0.       0.     ]
  [ 0.       0.       0.      ...  0.       0.       0.     ]
  [ 0.       0.       0.      ...  0.       0.       0.     ]
  ...
  [ 1.6624   1.4764   1.4435  ...  2.1533  15.271   21.838  ]
  [ 1.7058   1.5209   1.6649  ...  2.3547  17.145   23.364  ]
  [ 1.7058   1.6991   1.7581  ...  2.4202  17.783   24.029  ]]

 [[ 0.       0.       0.      ...  0.       0.       0.     ]
  [ 0.       0.       0.      ...  0.       0.       0.     ]
  [ 0.       0.       0.      ...  0.       0.       0.     ]
  ...
  [ 1.4018   1.3873   1.4085  ...  2.2893  17.145   23.808  ]
  [ 1.4886   1.5209   1.56    ...  2.3799  17.996   24.497  ]
  [ 1.5755   1.4764   1.6066  ...  2.4051  18.209   24.768  ]]

 [[ 0.       0.       0.      ...  0.       0.       0.     ]
  [ 0.       0.       0.      ...  0.       0.       0.     ]
  [ 0.       0.       0.      ...  0.       0.       0.     ]
  ...
  [ 1.0977   1.1646   1.1988  ...  2.0476  15.48

In [16]:
print(output_trainset)

[]


In [25]:
print(output_train)

[array([[1.e+00, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00],
       [1.e+04, 1.e+04, 1.e+04, ..., 1.e+04, 1.e+04, 1.e+04],
       [1.e+00, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 0.e+00],
       ...,
       [1.e+04, 1.e+04, 1.e+04, ..., 1.e+04, 1.e+04, 1.e+04],
       [0.e+00, 0.e+00, 0.e+00, ..., 0.e+00, 0.e+00, 1.e+00],
       [1.e+04, 1.e+04, 1.e+04, ..., 1.e+04, 1.e+04, 1.e+04]])]


In [26]:
print(type(output_train))

<class 'list'>


In [27]:
t = np.array(output_train)
print(t)

[[[1.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  [1.e+04 1.e+04 1.e+04 ... 1.e+04 1.e+04 1.e+04]
  [1.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 0.e+00]
  ...
  [1.e+04 1.e+04 1.e+04 ... 1.e+04 1.e+04 1.e+04]
  [0.e+00 0.e+00 0.e+00 ... 0.e+00 0.e+00 1.e+00]
  [1.e+04 1.e+04 1.e+04 ... 1.e+04 1.e+04 1.e+04]]]


In [28]:
print(type(t))

<class 'numpy.ndarray'>


In [29]:
print(t.shape)

(1, 1760, 11)


In [5]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

RuntimeError: Physical devices cannot be modified after being initialized